Q1.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

np.random.seed(42)

n_samples = 1000

X1 = np.random.rand(n_samples)
X2 = X1 + np.random.normal(0, 0.01, n_samples)
X3 = X1 + np.random.normal(0, 0.01, n_samples)
X4 = X2 + np.random.normal(0, 0.01, n_samples)
X5 = X3 + np.random.normal(0, 0.01, n_samples)
X6 = X5 + np.random.normal(0, 0.01, n_samples)
X7 = X4 + np.random.normal(0, 0.01, n_samples)

X = np.stack([X1, X2, X3, X4, X5, X6, X7], axis=1)
y = 3 * X1 + 2 * X2 + 0.5 * X7 + np.random.normal(0, 0.1, n_samples)

df = pd.DataFrame(X, columns=[f'X{i}' for i in range(1, 8)])
df['y'] = y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Number of NaN values in X_train:", np.isnan(X_train_scaled).sum())
print("Number of NaN values in X_test:", np.isnan(X_test_scaled).sum())
print("Number of NaN values in y_train:", np.isnan(y_train).sum())
print("Number of NaN values in y_test:", np.isnan(y_test).sum())
if (np.isnan(X_train_scaled).sum() > 0 or np.isnan(y_train).sum() > 0):
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy="mean")

    X_train_scaled = imputer.fit_transform(X_train_scaled)
    X_test_scaled = imputer.transform(X_test_scaled)
    y_train = imputer.fit_transform(y_train.reshape(-1, 1)).ravel()
    y_test = imputer.transform(y_test.reshape(-1, 1)).ravel()
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
regularization_params = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]
best_r2 = -float("inf")
best_lr = None
best_reg = None
best_cost = None
class RidgeRegressionGD:
    def __init__(self, learning_rate=0.01, reg_param=1, n_iters=500):
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.n_iters = n_iters

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            dw = (1/m) * np.dot(X.T, (y_pred - y)) + 2 * self.reg_param * self.weights
            db = (1/m) * np.sum(y_pred - y)
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def compute_cost(self, X, y):
        m = len(y)
        y_pred = np.dot(X, self.weights) + self.bias
        cost = (1 / (2 * m)) * np.sum((y_pred - y) ** 2) + self.reg_param * np.sum(self.weights ** 2)
        return cost

for lr in learning_rates:
    for reg in regularization_params:
        model = RidgeRegressionGD(learning_rate=lr, reg_param=reg, n_iters=1000)
        model.fit(X_train_scaled, y_train)

        predictions = model.predict(X_test_scaled)
        if np.isnan(predictions).any():
            continue

        r2 = r2_score(y_test, predictions)
        cost = model.compute_cost(X_test_scaled, y_test)

        print(f"Learning Rate: {lr}, Regularization: {reg}, R2_score: {r2:.4f}, Cost: {cost:.4f}")

        if r2 > best_r2:
            best_r2 = r2
            best_lr = lr
            best_reg = reg
            best_cost = cost
best_lr, best_reg, best_r2, best_cost


Number of NaN values in X_train: 0
Number of NaN values in X_test: 0
Number of NaN values in y_train: 0
Number of NaN values in y_test: 0
Learning Rate: 0.0001, Regularization: 1e-15, R2_score: -1.5199, Cost: 3.1713
Learning Rate: 0.0001, Regularization: 1e-10, R2_score: -1.5199, Cost: 3.1713
Learning Rate: 0.0001, Regularization: 1e-05, R2_score: -1.5199, Cost: 3.1713
Learning Rate: 0.0001, Regularization: 0.001, R2_score: -1.5199, Cost: 3.1714
Learning Rate: 0.0001, Regularization: 0, R2_score: -1.5199, Cost: 3.1713
Learning Rate: 0.0001, Regularization: 1, R2_score: -1.5518, Cost: 3.2901
Learning Rate: 0.0001, Regularization: 10, R2_score: -1.7776, Cost: 3.7115
Learning Rate: 0.0001, Regularization: 20, R2_score: -1.9042, Cost: 3.8157
Learning Rate: 0.001, Regularization: 1e-15, R2_score: 0.6017, Cost: 0.5013
Learning Rate: 0.001, Regularization: 1e-10, R2_score: 0.6017, Cost: 0.5013
Learning Rate: 0.001, Regularization: 1e-05, R2_score: 0.6017, Cost: 0.5013
Learning Rate: 0.001, Re

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1220: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
<ipython-input-9-d5a8b3cd173b>:58: RuntimeWarning: overflow encountered in square
  cost = (1 / (2 * m)) * np.sum((y_pred - y) ** 2) + self.reg_param * np.sum(self.weights ** 2)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-9-d5a8b3cd173b>:47: RuntimeWarning: invalid value encountered in multiply
  dw = (1/m) * np.dot(X.T, (y_pred - y)) + 2 * self.reg_param * self.weights


(0.1, 1e-15, 0.995876532335217, 0.005189314144506638)

Q2.

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
hitters_df = pd.read_csv("Hitters.csv")
print(hitters_df.isnull().sum())
num_imputer = SimpleImputer(strategy='mean')
hitters_df['Salary'] = num_imputer.fit_transform(hitters_df[['Salary']])
hitters_df.dropna(inplace=True)
categorical_columns = hitters_df.select_dtypes(include=['object']).columns
hitters_df = pd.get_dummies(hitters_df, columns=categorical_columns, drop_first=True)
print(hitters_df.info())

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AtBat        322 non-null    int64  
 1   Hits         322 non-null    int64  
 2   HmRun        322 non-null    int64  
 3   Runs         322 non-null    int64  
 4   RBI          322 non-null    int64  
 5   Walks        322 non-null    int64  
 6   Years        322 non-null    int64  
 7   CAtBat       322 non-null    int64  
 8   CHits        322 non-null    int64  
 9   CHmRun       322 non-null    int64  
 10  CRuns        322 non-null    int64  
 11  CRBI       

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = hitters_df.drop(columns=['Salary'])
y = hitters_df['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"X_train shape: {X_train_scaled.shape}")
print(f"X_test shape: {X_test_scaled.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (257, 19)
X_test shape: (65, 19)
y_train shape: (257,)
y_test shape: (65,)


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
ridge_model = Ridge(alpha=0.5748)
ridge_model.fit(X_train_scaled, y_train)
lasso_model = Lasso(alpha=0.5748)
lasso_model.fit(X_train_scaled, y_train)
print("Linear Regression Coefficients:", linear_model.coef_)
print("Ridge Regression Coefficients:", ridge_model.coef_)
print("LASSO Regression Coefficients:", lasso_model.coef_)

Linear Regression Coefficients: [-280.34211773  254.85642954   51.06688158  -23.32191325  -51.72982757
  109.44151762  -39.64710488 -560.66509814  407.02049119  -11.01674092
  297.20460427  286.33983861 -205.9936606    51.68994142   44.23399217
  -20.90083424   18.6843613   -50.6208016   -12.24371842]
Ridge Regression Coefficients: [-290.55944588  258.90121297   39.83326588  -20.25178104  -37.74330697
  109.98327779  -60.76017277 -325.36790562  255.74410895    6.28951331
  267.6023989   241.2276548  -214.05273468   52.30163036   39.60185055
  -20.03254578   16.05457488  -52.30078515  -10.87485569]
LASSO Regression Coefficients: [-283.34161179  242.81113277   32.70143979   -8.92295822  -28.44998553
  103.98988805  -52.21848263 -383.07585691  291.26643367    0.
  266.86780871  252.83100034 -203.67909711   51.83703983   38.53611158
  -18.40175814   11.58836304  -51.06696766   -5.84414368]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.032e+03, tolerance: 4.096e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
linear_predictions = linear_model.predict(X_test_scaled)
ridge_predictions = ridge_model.predict(X_test_scaled)
lasso_predictions = lasso_model.predict(X_test_scaled)
linear_r2 = r2_score(y_test, linear_predictions)
linear_mse = mean_squared_error(y_test, linear_predictions)
ridge_r2 = r2_score(y_test, ridge_predictions)
ridge_mse = mean_squared_error(y_test, ridge_predictions)
lasso_r2 = r2_score(y_test, lasso_predictions)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
print(f"Linear Regression - R^2: {linear_r2:.4f}, MSE: {linear_mse:.4f}")
print(f"Ridge Regression - R^2: {ridge_r2:.4f}, MSE: {ridge_mse:.4f}")
print(f"LASSO Regression - R^2: {lasso_r2:.4f}, MSE: {lasso_mse:.4f}")
if linear_r2 > ridge_r2 and linear_r2 > lasso_r2:
    print("Linear Regression performs the best.")
elif ridge_r2 > linear_r2 and ridge_r2 > lasso_r2:
    print("Ridge Regression performs the best.")
else:
    print("LASSO Regression performs the best.")

Linear Regression - R^2: 0.3369, MSE: 124847.6947
Ridge Regression - R^2: 0.3521, MSE: 121982.3621
LASSO Regression - R^2: 0.3519, MSE: 122022.6426
Ridge Regression performs the best.


Q3.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
alphas = np.logspace(-6, 6, 13)
ridge_cv = RidgeCV(alphas=alphas, scoring='r2', cv=5)
ridge_cv.fit(X_train, y_train)
y_pred_ridge = ridge_cv.predict(X_test)
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)
y_pred_lasso = lasso_cv.predict(X_test)
ridge_r2 = r2_score(y_test, y_pred_ridge)
lasso_r2 = r2_score(y_test, y_pred_lasso)
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
lasso_mse = mean_squared_error(y_test, y_pred_lasso)
print(f"RidgeCV - Best Alpha: {ridge_cv.alpha_}, R²: {ridge_r2:.4f}, MSE: {ridge_mse:.4f}")
print(f"LassoCV - Best Alpha: {lasso_cv.alpha_}, R²: {lasso_r2:.4f}, MSE: {lasso_mse:.4f}")

RidgeCV - Best Alpha: 1e-06, R²: 0.5758, MSE: 0.5559
LassoCV - Best Alpha: 0.001, R²: 0.5769, MSE: 0.5545
